## note on 2020.1.14
- storing temporary data locally significantly increase processing speed. finish in 90 minutes using 4 parallel thread (python scripts)
- uint8 bit has data folding problem, will need to use float16 for higher requirement
- check missing data: 260 

In [1]:
import glob
import numpy as np
from datetime import datetime,timedelta
import calendar
from netCDF4 import Dataset
import joblib
import os
print(os.getcwd())

data_folder = "/Volumes/own4T/jma_radar_401300210"
output_folder = "/Users/jiang/data/jma_radar"

dt     = datetime(2019,4,1,0,0)  # included
dt_end = datetime(2019,7,1,0,5)  # not included
time_step = 5 * 60 # seconds

# initialize
#current_month = dt.month  # when change to new month, store previous month's data
#_, days_in_month = calendar.monthrange(dt.year, dt.month)
print(f"today: {dt}")
today = dt.day
daily_data = np.zeros(shape = (288,1000,1000), dtype = np.uint8)
output_file  = f"jma_radar_uint8_{dt.strftime('%Y_%m_%d')}.joblib"
output_path = os.path.join(output_folder, output_file)


while dt <= dt_end:
    # if new day store data and reset counter                     
    if dt.day != today:
        joblib.dump(daily_data, output_path)
        del daily_data            

        # reset
        print(f"today: {dt}")
        today = dt.day
        daily_data = np.zeros(shape = (288,1000,1000), dtype = np.uint8)
        output_file  = f"jma_radar_uint8_{dt.strftime('%Y_%m_%d')}.joblib"
        output_path = os.path.join(output_folder, output_file)
        
    # grib2 file to nc file
    grib_file = dt.strftime('%Y%m%d_%H%M00.000')
    source_folder = os.path.join(data_folder, dt.strftime("%Y/%m/%d"))
    source_path = os.path.join(source_folder, grib_file)
    nc_file = "temp1.nc"
    cmd = f"wgrib2 {source_path} -s | egrep 'surface'|wgrib2 -i {source_path} -netcdf {nc_file}"
    fail = os.system(cmd)
    if fail:
        print("wgrib2 wrong at ",source_path)
        dt += timedelta(seconds = time_step)
        continue
    
    # nc file to data
    #root = Dataset(nc_file, "r")
    #rain = root['var0_1_203_surface'][0,:,:] # masked array, shape(3360,3560)
    #rain_reduced = rain[1500:2500,1000:2000].copy()
    rain_reduced = Dataset(nc_file, "r")['var0_1_203_surface'][0,1500:2500,1000:2000]
    rain_reduced.fill_value = 0.0                    
    #rain_filled = rain_reduced.filled() 
    cnt = int(dt.hour*12 + (dt.minute)/5)                   
    daily_data[cnt] = (rain_reduced.filled() * 10).astype('uint8')                     
    
    # delete nc                     
    os.system(f"rm -r {nc_file}")                    
    dt += timedelta(seconds = time_step)

/Users/jiang/github_all/rainymotion/docs/notebooks
today: 2019-04-01 00:00:00
wgrib2 wrong at  /Volumes/own4T/jma_radar_401300210/2019/04/01/20190401_235500.000
today: 2019-04-02 00:00:00
today: 2019-04-03 00:00:00
today: 2019-04-04 00:00:00
today: 2019-04-05 00:00:00
today: 2019-04-06 00:00:00
today: 2019-04-07 00:00:00
today: 2019-04-08 00:00:00
wgrib2 wrong at  /Volumes/own4T/jma_radar_401300210/2019/04/08/20190408_150000.000
wgrib2 wrong at  /Volumes/own4T/jma_radar_401300210/2019/04/08/20190408_150500.000
wgrib2 wrong at  /Volumes/own4T/jma_radar_401300210/2019/04/08/20190408_151000.000
wgrib2 wrong at  /Volumes/own4T/jma_radar_401300210/2019/04/08/20190408_151500.000
wgrib2 wrong at  /Volumes/own4T/jma_radar_401300210/2019/04/08/20190408_152000.000
wgrib2 wrong at  /Volumes/own4T/jma_radar_401300210/2019/04/08/20190408_152500.000
wgrib2 wrong at  /Volumes/own4T/jma_radar_401300210/2019/04/08/20190408_153000.000
wgrib2 wrong at  /Volumes/own4T/jma_radar_401300210/2019/04/08/201904

### check all missing files
- total missing: 260, equivalent to one day missing, so it doesn't matter

In [3]:
data_folder = "/Volumes/own4T/jma_radar_401300210"
dt     = datetime(2019,1,1,0,0)  # included
dt_end = datetime(2020,1,1,0,5)  # not included
time_step = 5 * 60 # seconds
cnt = 0
while dt <= dt_end:
    grib_file = dt.strftime('%Y%m%d_%H%M00.000')
    source_folder = os.path.join(data_folder, dt.strftime("%Y/%m/%d"))
    source_path = os.path.join(source_folder, grib_file)
    if os.path.isfile(source_path) == False:
        print(grib_file)
        cnt += 1
    
    dt += timedelta(seconds = time_step)
print(f"total missing files: {cnt}") 

20190110_061500.000
20190401_235500.000
20190408_150000.000
20190408_150500.000
20190408_151000.000
20190408_151500.000
20190408_152000.000
20190408_152500.000
20190408_153000.000
20190408_153500.000
20190408_154000.000
20190408_155000.000
20190408_160000.000
20190408_160500.000
20190408_161000.000
20190408_162000.000
20190408_163500.000
20190408_164000.000
20190408_164500.000
20190505_220000.000
20190505_220500.000
20190505_221000.000
20190505_221500.000
20190505_222000.000
20190505_222500.000
20190505_223000.000
20190505_223500.000
20190505_224000.000
20190505_224500.000
20190505_225000.000
20190505_225500.000
20190505_230000.000
20190505_230500.000
20190505_231000.000
20190505_231500.000
20190505_232000.000
20190505_232500.000
20190505_233000.000
20190505_233500.000
20190505_234000.000
20190505_234500.000
20190505_235000.000
20190505_235500.000
20190506_000000.000
20190615_010500.000
20190615_011000.000
20190615_011500.000
20190615_125500.000
20191107_075000.000
20191107_075500.000
